In [2]:
!pip install nltk

                                              0.0/1.5 MB ? eta -:--:--
     -------                                  0.3/1.5 MB 5.7 MB/s eta 0:00:01
     -----------------                        0.7/1.5 MB 6.9 MB/s eta 0:00:01
     ---------------------------              1.0/1.5 MB 7.3 MB/s eta 0:00:01
     -------------------------------------    1.4/1.5 MB 7.6 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 7.4 MB/s eta 0:00:00


In [4]:
import os                         # os is a module for navigating your machine (e.g., file directories).
import nltk                       # nltk stands for natural language tool kit and is useful for text-mining. 
from nltk import word_tokenize    # and some of its key functions
from nltk import sent_tokenize    
import numpy as np
import pandas as pd
import csv                        # csv is for importing and working with csv files
import statistics
import datetime
date = datetime.date.today()

import re                         # things we need for RegEx corrections

In [5]:
os.listdir("..\\processed_dataset_text")  

['ESHG2001.txt',
 'ESHG2001abstractICHG.txt',
 'ESHG2002Abstracts.txt',
 'ESHG2012Abstracts.txt',
 'ESHG2013Abstracts.txt',
 'ESHG2014Abstracts.txt',
 'ESHG2015Abstracts.txt',
 'ESHG2016Abstracts.txt']

In [ ]:
def match_strings_in_text(input, output, list_of_strings):
    list_of_results = []
    # Open the file in read only mode
    for filename in os.listdir(input):
        name = filename.replace(r'.txt', "")
        line_number = 0
        with open(input + "\\" + filename, "r", encoding='ISO-8859-1') as read_obj:
            for line in read_obj:
                line_number += 1
                # For each line, check if line contains any string from the list of strings
                for string_to_search in list_of_strings:
                    if string_to_search in line:
                    # If any string is found in line, then append that line along with line number in list
                        list_of_results.append((name, string_to_search, line_number, line.rstrip()))
    no_dups_results = list(set(list_of_results))
    with open(output + "\\select.csv", "w", encoding='ISO-8859-1') as outfile:
        write = csv.writer(outfile)
        write.writerows(no_dups_results)



In [ ]:
 match_strings_in_text('..\\for_analysis\\ESHG', "..\\counts\\ESHG", ['autis', 'Autis', 'ASD', 'Asperger', 'asperger',])